In [1]:
# REMEMBER TO ACTIVATE THE CONDA ENV FIRST

! pip install numpy
! pip install scipy
! pip install matplotlib
! pip install sklearn
! pip install scikit-learn


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
! pip install tqdm


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# Import relevant components

import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tqdm.notebook import trange, tqdm

In [4]:
from collections import defaultdict
import random

# Look at how the data is stored

In [51]:
embeddings_dict = {}
top_5000_common_words = []
with open("glove.6B/glove.6B.300d.txt", 'r', encoding="utf-8") as f:
    for index, line in enumerate(f):
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector
        if index < 5000:
            top_5000_common_words.append(word)

In [13]:
# Calculate the average length of a vector

distance_sums = 0
for word, embedding in embeddings_dict.items():
    distance_sums += np.linalg.norm(embedding)
average_distance = distance_sums / len(embeddings_dict)

In [14]:
print(average_distance)

4.474557736493945


In [20]:
# For simplicity

def distance(word1, word2 = ''):
    if not word2:
        return np.linalg.norm(embeddings_dict[word1])
    else:
        return np.linalg.norm(embeddings_dict[word1] - embeddings_dict[word2])

# Work with individual words

In [22]:
print(distance('dog'))
print(distance('dogs'))
print(distance('dog', 'dogs'))

4.8580456
4.866664
2.6515272


In [28]:
# Given two words, return the index where the two arrays are most different
def max_difference(word1, word2):
    return np.argmax(np.abs(embeddings_dict[word1] - embeddings_dict[word2]))

In [40]:
biggest_difference = defaultdict(int)
for word, embedding in embeddings_dict.items():
    if (word + 's') in embeddings_dict:
        biggest_difference[max_difference(word, word + 's')] += 1
    elif (word + 'es') in embeddings_dict:
        biggest_difference[max_difference(word, word + 'es')] += 1
sorted_differences = sorted(biggest_difference.items(), key=(lambda x: x[1]), reverse=True)
print(sorted_differences)

[(30, 4105), (25, 2241), (31, 1639), (22, 1100), (42, 1069), (4, 953), (43, 910), (19, 895), (8, 868), (32, 811), (48, 782), (49, 758), (5, 724), (26, 709), (38, 680), (0, 676), (3, 664), (13, 649), (33, 624), (18, 621), (29, 600), (44, 587), (12, 583), (1, 579), (45, 564), (17, 554), (2, 522), (37, 507), (39, 502), (47, 501), (9, 492), (16, 483), (41, 462), (20, 456), (7, 449), (6, 433), (28, 430), (40, 412), (24, 391), (21, 390), (34, 376), (14, 349), (11, 335), (27, 331), (15, 318), (46, 279), (23, 275), (36, 266), (35, 235), (10, 215)]


In [39]:
from scipy.optimize import minimize_scalar

def loss(n):
    # Calculate the loss for a given value of n
    loss = 0
    for i in range(1, 50):
        predicted_value = n / i
        actual_value = sorted_differences[i - 1][1]
        loss += abs(predicted_value - actual_value)
    return loss

# Find the value of n that minimizes the loss
result = minimize_scalar(loss, method='bounded', bounds=(0.1, 1000))

print(f"Optimal n: {result.x}")
print(f"Minimum loss: {result.fun}")


Optimal n: 999.9999700496764
Minimum loss: 29659.794795824222


# Nothing here! 

Time to look at the values for each index 

In [46]:
overall_differences_vector = np.zeros(embeddings_dict['the'].shape[0])
for word, embedding in embeddings_dict.items():
    if (word + 's') in embeddings_dict:
        overall_differences_vector += np.abs(embeddings_dict[word] - embeddings_dict[word + 's'])
    elif (word + 'es') in embeddings_dict:
        overall_differences_vector += np.abs(embeddings_dict[word] - embeddings_dict[word + 'es'])
reversed_indeces = np.argsort(overall_differences_vector)[::-1]
print([(reversed_index, round(overall_differences_vector[reversed_index])) for reversed_index in reversed_indeces])

[(30, 26727), (25, 25177), (31, 21030), (42, 19360), (43, 19345), (22, 19150), (4, 19105), (19, 18874), (8, 18416), (49, 18328), (48, 18106), (5, 18094), (32, 18085), (33, 18074), (38, 18065), (18, 18011), (0, 17963), (3, 17851), (12, 17785), (26, 17750), (1, 17690), (29, 17609), (13, 17595), (17, 17592), (44, 17337), (41, 17279), (39, 17269), (45, 17263), (47, 17196), (2, 17056), (6, 17047), (37, 16940), (7, 16861), (20, 16791), (16, 16673), (40, 16647), (28, 16627), (9, 16557), (14, 16517), (24, 16505), (21, 16337), (34, 16170), (27, 15923), (11, 15780), (15, 15609), (35, 15575), (46, 15470), (23, 15398), (36, 15313), (10, 15049)]


# Ok still nothing :/

But what about past tense

In [59]:
def likely_s_adding_method(word):
    if word[-1] == 's' or word[-1] == 'z' or word[-1] == 'x' or (len(word) > 1 and (word[-2:] == 'sh' or word[-2:] == 'ch')):
        return word + 'es'
    return word + 's'
        

In [60]:
def is_probably_a_verb(word):
    return word + 'ing' in embeddings_dict and word + 'ed' in embeddings_dict and likely_s_adding_method(word) in embeddings_dict

In [63]:
biggest_difference_past = defaultdict(int)
biggest_difference_present = defaultdict(int)
biggest_difference_3sg = defaultdict(int)
for word, embedding in embeddings_dict.items():
    if is_probably_a_verb(word):
        biggest_difference_past[max_difference(word, word + 'ed')] += 1
        biggest_difference_present[max_difference(word, word + 'ing')] += 1
        biggest_difference_3sg[max_difference(word, likely_s_adding_method(word))] += 1
sorted_differences_past = sorted(biggest_difference_past.items(), key=(lambda x: x[1]), reverse=True)
sorted_differences_present = sorted(biggest_difference_present.items(), key=(lambda x: x[1]), reverse=True)
sorted_differences_3sg = sorted(biggest_difference_3sg.items(), key=(lambda x: x[1]), reverse=True)
print("past: ", sorted_differences_past)
print("present: ", sorted_differences_present)
print("3sg: ", sorted_differences_3sg)

past:  [(30, 646), (32, 194), (31, 106), (49, 94), (26, 90), (1, 88), (46, 82), (27, 78), (6, 76), (37, 71), (25, 61), (18, 60), (21, 59), (16, 55), (12, 53), (47, 53), (38, 50), (15, 49), (19, 49), (4, 43), (28, 42), (11, 41), (3, 40), (22, 40), (44, 37), (9, 36), (29, 30), (7, 29), (13, 28), (20, 27), (8, 27), (5, 27), (0, 26), (39, 24), (43, 24), (23, 23), (45, 23), (48, 22), (36, 22), (2, 20), (33, 20), (40, 17), (14, 16), (34, 15), (42, 14), (17, 14), (24, 14), (35, 12), (10, 12), (41, 7)]
present:  [(30, 624), (15, 155), (32, 133), (25, 113), (26, 85), (49, 76), (29, 70), (1, 68), (16, 68), (2, 65), (18, 58), (27, 56), (31, 55), (39, 53), (19, 51), (47, 51), (11, 50), (21, 49), (3, 41), (4, 41), (33, 41), (38, 40), (17, 38), (0, 38), (28, 37), (45, 37), (12, 36), (43, 36), (9, 34), (5, 32), (6, 30), (37, 29), (22, 28), (7, 28), (14, 27), (13, 27), (8, 26), (44, 25), (48, 24), (20, 24), (24, 24), (40, 24), (36, 23), (23, 21), (46, 20), (35, 19), (42, 17), (34, 17), (41, 13), (10, 

# Ok last thing before I sleep

Is it just that 30 is use more often???? Idrk what this even means but does it really represent inflectional morphology?

In [79]:
biggest_random_difference = defaultdict(int)
for i in range(10000):
    random_keys = random.sample(embeddings_dict.keys(), 2)
    biggest_random_difference[max_difference(random_keys[0], random_keys[1])] += 1
    if i % 100 == 0:
        print(i)
sorted_random_differences = sorted(biggest_random_difference.items(), key=(lambda x: x[1]), reverse=True)
print(sorted_random_differences)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
[(30, 664), (26, 321), (13, 312), (5, 302), (11, 300), (8, 297), (4, 260), (0, 248), (33, 247), (20, 247), (12, 230), (49, 224), (47, 222), (18, 222), (6, 220), (28, 219), (3, 218), (9, 211), (42, 205), (43, 204), (38, 202), (45, 199), (25, 199), (44, 197), (2, 194), (37, 193), (1, 192), (39, 191), (48, 186), (16, 182), (41, 179), (7, 174), (19, 173), (14, 165), (22, 161), (31, 159), (21, 155), (24, 152), (29, 145), (23, 144), (32, 143), (34, 138), (17, 129), (40, 113), (46, 108), (10, 108), (15, 105), (35,

In [87]:
# Ok but is it just that it's the biggest????

biggest_index = defaultdict(int)
for word, embedding in embeddings_dict.items():
    biggest_index[np.argmax(np.abs(embedding))] += 1
sorted_biggest_indeces = sorted(biggest_index.items(), key=(lambda x: x[1]), reverse=True)
print(sorted_biggest_indeces)

[(30, 95612), (25, 32342), (6, 12237), (12, 10977), (26, 9630), (18, 9443), (1, 9009), (33, 8669), (5, 8478), (4, 8067), (8, 7457), (13, 7450), (11, 7249), (9, 7048), (14, 7015), (0, 6630), (43, 6203), (20, 6128), (45, 6061), (47, 5992), (49, 5683), (38, 5630), (28, 5598), (3, 5581), (19, 5549), (7, 5535), (41, 5388), (44, 5384), (42, 4843), (32, 4815), (24, 4650), (22, 4636), (2, 4607), (39, 4607), (37, 4595), (46, 4487), (48, 4352), (31, 4300), (16, 4245), (17, 4164), (21, 3930), (40, 3809), (29, 3570), (34, 3467), (23, 3269), (15, 2686), (27, 2685), (10, 2255), (36, 2011), (35, 1972)]


# Ok well now I need to redo it all but normalized this is some bs

Ok clearly my earlier tests weren't working today is Dec 23 2023 and I want results 

In [6]:
# Define the similarity between two vectors as the cosine similarity
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)

    similarity = dot_product / (norm_vector1 * norm_vector2)

    return similarity

In [31]:
# Get the most similar vector to another one
def most_similar(raw_vector, embeddings_dict, top_k):
    cosine_similarities = []
    for word in embeddings_dict:
        cosine_similarities.append((word, cosine_similarity(raw_vector, embeddings_dict[word] ) ) )
    # Sort based on the cosine similarity
    cosine_similarities = sorted(cosine_similarities, key=lambda x: x[1], reverse=True)
    return cosine_similarities[:top_k]

In [42]:
def length(vector):
    return np.linalg.norm(vector)

In [32]:
print(most_similar(embeddings_dict["test"], embeddings_dict, 5))

[('test', 1.0), ('tests', 0.91643566), ('testing', 0.81992215), ('tested', 0.74428755), ('final', 0.69102275)]


In [75]:
# Get three random words and operate on them. Then, find the closest words
vector1 = random.choice(top_5000_common_words)
vector2 = random.choice(top_5000_common_words)
vector3 = random.choice(top_5000_common_words)
print(f"{vector1} - {vector2} + {vector3}")
most_similar_words = most_similar(embeddings_dict[vector1] - embeddings_dict[vector2] + embeddings_dict[vector3], embeddings_dict, 5)
print(most_similar_words)

tall - october + campaigns
[('tall', 0.64718676), ('campaigns', 0.5039532), ('taller', 0.39721638), ('erect', 0.38956788), ('ads', 0.38947594)]


In [72]:
# Get three not random words and operate on them. Then, find the closest words
# dance - dancer = sing - singer
# dance - dancer - sing = -singer
# -dance + dancer + sing = singer
vector1 = "dance"
vector2 = "dancer"
vector3 = "sing"
print(length(embeddings_dict[vector1]))
print(length(embeddings_dict[vector2]))
print(length(embeddings_dict[vector3]))
print(f"{vector1} - {vector2} + {vector3}")
most_similar_words = most_similar(embeddings_dict[vector1] - embeddings_dict[vector2] + embeddings_dict[vector3], embeddings_dict, 5)
print(most_similar_words)

6.8651047
6.5954485
6.8510113
dance - dancer + sing
[('sing', 0.808328), ('dance', 0.6101845), ('singing', 0.59580183), ('songs', 0.5877572), ('tune', 0.53933567)]


In [78]:
# Get three not random words and operate on them. Then, find the closest words
# dance - dancer = sing - singer
# dance - dancer - sing = -singer
# -dance + dancer + sing = singer
vector1 = "dance"
vector2 = "dancer"
vector3 = "sing"
print(length(embeddings_dict[vector1]))
print(length(embeddings_dict[vector2]))
print(length(embeddings_dict[vector3]))
print(f"{vector1} is to {vector2} as {vector3} is to")
most_similar_words = most_similar(0 - embeddings_dict[vector1] + embeddings_dict[vector2] + embeddings_dict[vector3], embeddings_dict, 10)
print(most_similar_words)

6.8651047
6.5954485
6.8510113
dance is to dancer as sing is to
[('sing', 0.6812048), ('dancer', 0.55380404), ('sang', 0.53223425), ('sings', 0.5071834), ('singer', 0.49010777), ('baritone', 0.4507536), ('singers', 0.44889233), ('soprano', 0.44633728), ('singing', 0.44221634), ('nyah', 0.42972228)]
